In [1]:
import gc
import re
import sys
import time
#import jieba
import string
import codecs
import pickle
import hashlib
import os.path
import datetime
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost
import matplotlib
import matplotlib.pyplot as plt
import seaborn 
#from datetime import datetime
from collections import Counter
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder  
from itertools import groupby 

/home/xiaoran/Tools/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
orderFuture_train = pd.read_csv('../data/train/orderFuture_train.csv')#0-33682  1-6625  all-40307
orderFuture_test  = pd.read_csv('../data/test/orderFuture_test.csv')#10076 #6625/40307*10076=1656.127

orderFuture_test['orderType'] = -1
data = pd.concat([orderFuture_train,orderFuture_test])
data.columns = ['userid', 'label']

In [11]:
userProfile_train  = pd.read_csv('../data/train/userProfile_train.csv')
userProfile_test   = pd.read_csv('../data/test/userProfile_test.csv')

userProfile = pd.concat([userProfile_train,userProfile_test]).fillna('unknow')



In [12]:
action_train  = pd.read_csv('../data/train/action_train.csv')
action_test   = pd.read_csv('../data/test/action_test.csv')

action_train['time_rank'] = action_train['actionTime'].groupby(action_train['userid']).rank(ascending=False,method='first')
action_test['time_rank']  = action_test['actionTime'].groupby(action_test['userid']).rank(ascending=False,  method='first')
action =  pd.concat([action_train,action_test])


In [13]:
orderHistory_train  = pd.read_csv('../data/train/orderHistory_train.csv')
orderHistory_test   = pd.read_csv('../data/test/orderHistory_test.csv')
orderHistory_train =  pd.concat([orderHistory_train,orderHistory_test])

userComment_train  = pd.read_csv("../data/train/userComment_train.csv")
userComment_test   = pd.read_csv("../data/test/userComment_test.csv")
userComment =  pd.concat([userComment_train,userComment_test])


In [14]:
#######################################feature#######################################
################ 1、userProfile
class_le = LabelEncoder()  
userProfile['gender']   = class_le.fit_transform(userProfile['gender'].values)
userProfile['province'] = class_le.fit_transform(userProfile['province'].values)
userProfile['age']      = class_le.fit_transform(userProfile['age'].values)
data = data.merge(userProfile, on='userid', how='left')

#one-hot
feat = userProfile[['userid','gender','age']]
feat['gender'] = ['_{0}'.format(i) for i in feat['gender']]
feat['age']    = ['_{0}'.format(i)    for i in feat['age']]
feat = pd.get_dummies(feat)
data = data.merge(feat, on='userid', how='left')

#count
feat = pd.DataFrame(userProfile.groupby(['gender'])[['userid']].count()).reset_index()
feat.columns=["gender"]+["gender_cnt"]
data = data.merge(feat, on='gender', how='left')

feat = pd.DataFrame(userProfile.groupby(['age'])[['userid']].count()).reset_index()
feat.columns=["age"]+["age_cnt"]
data = data.merge(feat, on='age', how='left')

feat = pd.DataFrame(userProfile.groupby(['province'])[['userid']].count()).reset_index()
feat.columns=["province"]+["province_cnt"]
data = data.merge(feat, on='province', how='left')

#concat
data['age_province'] = data['age']*data['province']
data['age_gender'] = data['age']*data['gender']
data['province_gender'] = data['province']*data['gender']

#concat-count
feat = pd.DataFrame(data.groupby(['age_province'])[['userid']].count()).reset_index()
feat.columns=["age_province"]+["age_province_cnt"]
data = data.merge(feat, on='age_province', how='left')

feat = pd.DataFrame(data.groupby(['age_gender'])[['userid']].count()).reset_index()
feat.columns=["age_gender"]+["age_gender_cnt"]
data = data.merge(feat, on='age_gender', how='left')

feat = pd.DataFrame(data.groupby(['province_gender'])[['userid']].count()).reset_index()
feat.columns=["province_gender"]+["province_gender_cnt"]
data = data.merge(feat, on='province_gender', how='left')


/home/xiaoran/Tools/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/xiaoran/Tools/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
################ 2、action
action['actionType'] = pow(2,action['actionType'])


#all actionType
feat = pd.DataFrame(action.groupby(['userid'])[['actionType']].std()).reset_index()
feat.columns=["userid"]+["actionType_std"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(action.groupby(['userid'])[['actionType']].mean()).reset_index()
feat.columns=["userid"]+["actionType_mean"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(action.groupby(['userid'])[['actionType']].skew()).reset_index()
feat.columns=["userid"]+["actionType_skew"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(action.groupby(['userid'])[['actionType']].max()).reset_index()
feat.columns=["userid"]+["actionType_max"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(action.groupby(['userid'])[['actionType']].min()).reset_index()
feat.columns=["userid"]+["actionType_min"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(action.groupby(['userid'])[['actionType']].sum()).reset_index()
feat.columns=["userid"]+["actionType_sum"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(action.groupby(['userid'])[['actionType']].median()).reset_index()
feat.columns=["userid"]+["actionType_median"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(action.groupby(['userid'])[['actionType']].cumprod()).reset_index()
feat.columns=["userid"]+["actionType_cumprod"]
data = data.merge(feat, on='userid', how='left')

data['actionType_jc'] = data['actionType_max']-data['actionType_min']
data['actionType_cv'] = data['actionType_std']/data['actionType_mean']

#time_rank actionType
feat=pd.DataFrame(pd.pivot_table(action[['userid','time_rank','actionType']], index='userid', columns='time_rank')).reset_index()
feat = feat.iloc[:,:26]
feat.columns=["userid"]+["actionType_rank_{0}".format(i) for i in range(1,26)]
data = data.merge(feat, on='userid', how='left')

data['actionType_rank_diff_1'] = data['actionType_rank_1']-data['actionType_rank_2']
data['actionType_rank_diff_2'] = data['actionType_rank_2']-data['actionType_rank_3']
data['actionType_rank_diff_3'] = data['actionType_rank_3']-data['actionType_rank_4']
data['actionType_rank_diff_4'] = data['actionType_rank_4']-data['actionType_rank_5']
data['actionType_rank_diff_5'] = data['actionType_rank_5']-data['actionType_rank_6']
data['actionType_rank_diff_6'] = data['actionType_rank_6']-data['actionType_rank_7']
data['actionType_rank_diff_7'] = data['actionType_rank_7']-data['actionType_rank_8']
data['actionType_rank_diff_8'] = data['actionType_rank_8']-data['actionType_rank_9']
data['actionType_rank_diff_9'] = data['actionType_rank_9']-data['actionType_rank_10']
data['actionType_rank_diff_10'] = data['actionType_rank_10']-data['actionType_rank_11']


name = ["actionType_rank_diff_{0}".format(i) for i in range(1,11)]
feat['actionType_rank_diff_sum']  = data[name].apply(lambda x: x.sum(),axis=1)
feat['actionType_rank_diff_std']  = data[name].apply(lambda x: x.std(),axis=1)
feat['actionType_rank_diff_mean'] = data[name].apply(lambda x: x.mean(),axis=1)
data = data.merge(feat, on='userid', how='left')

action['actionTime_hour']  = action['actionTime'].fillna(0).apply(lambda x: time.localtime(int(x)).tm_hour)




#all actionTime
feat = pd.DataFrame(action.groupby(['userid'])[['actionTime']].std()).reset_index()
feat.columns=["userid"]+["actionTime_std"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(action.groupby(['userid'])[['actionTime']].mean()).reset_index()
feat.columns=["userid"]+["actionTime_mean"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(action.groupby(['userid'])[['actionTime']].skew()).reset_index()
feat.columns=["userid"]+["actionTime_skew"]
data = data.merge(feat, on='userid', how='left')


feat = pd.DataFrame(action.groupby(['userid'])[['actionTime_hour']].std()).reset_index()
feat.columns=["userid"]+["actionTime_hour_std"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(action.groupby(['userid'])[['actionTime_hour']].mean()).reset_index()
feat.columns=["userid"]+["actionTime_hour_mean"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(action.groupby(['userid'])[['actionTime_hour']].max()).reset_index()
feat.columns=["userid"]+["actionTime_hour_max"]
data = data.merge(feat, on='userid', how='left')


#all count
feat = pd.DataFrame(action.groupby(['userid'])[['actionTime']].count()).reset_index()
feat.columns=["userid"]+["action_cnt_all"]
data = data.merge(feat, on='userid', how='left')

#actionType count
feat = pd.DataFrame(pd.pivot_table(action,index=["userid"],columns='actionType',values=["actionTime"],aggfunc=len)).reset_index() 
feat.columns=["userid"]+["action_cnt_00{0}".format(i) for i in range(1,10)]
data = data.merge(feat, on='userid', how='left')

#actionType rate
data['action_001_rate'] = data['action_cnt_001'] /data['action_cnt_all']
data['action_002_rate'] = data['action_cnt_002'] /data['action_cnt_all']
data['action_003_rate'] = data['action_cnt_003'] /data['action_cnt_all']
data['action_004_rate'] = data['action_cnt_004'] /data['action_cnt_all']
data['action_005_rate'] = data['action_cnt_005'] /data['action_cnt_all']
data['action_006_rate'] = data['action_cnt_006'] /data['action_cnt_all']
data['action_007_rate'] = data['action_cnt_007'] /data['action_cnt_all']
data['action_008_rate'] = data['action_cnt_008'] /data['action_cnt_all']
data['action_009_rate'] = data['action_cnt_009'] /data['action_cnt_all']
data['action_1_4_rate'] = (data['action_cnt_001']+data['action_cnt_002']+
                           data['action_cnt_003']+data['action_cnt_004'])/data['action_cnt_all']
data['action_5_9_rate'] = (data['action_cnt_005']+data['action_cnt_006']+data['action_cnt_007']+
                           data['action_cnt_008']+data['action_cnt_009'])/data['action_cnt_all']

data['action_5_6_rate'] = data['action_005_rate'] +data['action_006_rate']
data['action_5_6_rara'] = data['action_005_rate'] /data['action_006_rate']

data['action_1_2_diff'] = data['action_cnt_001']-data['action_cnt_002']
data['action_2_3_diff'] = data['action_cnt_002']-data['action_cnt_003']
data['action_3_4_diff'] = data['action_cnt_003']-data['action_cnt_004']
data['action_4_5_diff'] = data['action_cnt_004']-data['action_cnt_005']
data['action_5_6_diff'] = data['action_cnt_005']-data['action_cnt_006']
data['action_6_7_diff'] = data['action_cnt_006']-data['action_cnt_007']
data['action_7_8_diff'] = data['action_cnt_007']-data['action_cnt_008']
data['action_8_9_diff'] = data['action_cnt_008']-data['action_cnt_009']
data['action_1_3_diff'] = data['action_cnt_001']-data['action_cnt_003']
data['action_5_9_diff'] = data['action_cnt_005']-data['action_cnt_009']


#max time all
feat = pd.DataFrame(action.groupby(['userid'])[['actionTime']].max()).reset_index()
feat.columns=["userid"]+["userid_actionTime_max"]
data = data.merge(feat, on='userid', how='left')

data['userid_actionTime_max_year']  = data['userid_actionTime_max'].apply(lambda x: time.localtime(int(x)).tm_year)
data['userid_actionTime_max_month'] = data['userid_actionTime_max'].apply(lambda x: time.localtime(int(x)).tm_mon)
data['userid_actionTime_max_hour']  = data['userid_actionTime_max'].apply(lambda x: time.localtime(int(x)).tm_hour)

#for i in data['userid_actionTime_max']: 
#    data['userid_actionTime_max_month'] = time.localtime(int(i)).tm_mon

#min time all
feat = pd.DataFrame(action.groupby(['userid'])[['actionTime']].max()).reset_index()
feat.columns=["userid"]+["userid_actionTime_min"]
data = data.merge(feat, on='userid', how='left')

data['userid_actionTime_last'] = data['userid_actionTime_max']-data['userid_actionTime_min']

#actionType max time
feat = pd.DataFrame(pd.pivot_table(action,index=["userid"],columns='actionType',values=["actionTime"],aggfunc=max)).reset_index() 
feat.columns=["userid"]+["actionTime_max_00{0}".format(i) for i in range(1,10)]
data = data.merge(feat, on='userid', how='left')

data['actionTime_max_005_year']  = data['actionTime_max_005'].fillna(0).apply(lambda x: time.localtime(int(x)).tm_year)
data['actionTime_max_005_month'] = data['actionTime_max_005'].fillna(0).apply(lambda x: time.localtime(int(x)).tm_mon)
data['actionTime_max_005_hour']  = data['actionTime_max_005'].fillna(0).apply(lambda x: time.localtime(int(x)).tm_hour)

data['actionTime_max_006_year']  = data['actionTime_max_006'].fillna(0).apply(lambda x: time.localtime(int(x)).tm_year)
data['actionTime_max_006_month'] = data['actionTime_max_006'].fillna(0).apply(lambda x: time.localtime(int(x)).tm_mon)
data['actionTime_max_006_hour']  = data['actionTime_max_006'].fillna(0).apply(lambda x: time.localtime(int(x)).tm_hour)

data['actionTime_max_007_year']  = data['actionTime_max_007'].fillna(0).apply(lambda x: time.localtime(int(x)).tm_year)
data['actionTime_max_007_month'] = data['actionTime_max_007'].fillna(0).apply(lambda x: time.localtime(int(x)).tm_mon)
data['actionTime_max_007_hour']  = data['actionTime_max_007'].fillna(0).apply(lambda x: time.localtime(int(x)).tm_hour)


data['actionTime_max_diff_9_8'] = data['actionTime_max_009']-data['actionTime_max_008']
data['actionTime_max_diff_8_7'] = data['actionTime_max_008']-data['actionTime_max_007']
data['actionTime_max_diff_7_6'] = data['actionTime_max_007']-data['actionTime_max_006']
data['actionTime_max_diff_6_5'] = data['actionTime_max_006']-data['actionTime_max_005']
data['actionTime_max_diff_5_4'] = data['actionTime_max_005']-data['actionTime_max_004']
data['actionTime_max_diff_9_7'] = data['actionTime_max_009']-data['actionTime_max_007']
data['actionTime_max_diff_9_6'] = data['actionTime_max_009']-data['actionTime_max_006']
data['actionTime_max_diff_9_5'] = data['actionTime_max_009']-data['actionTime_max_005']
data['actionTime_max_diff_9_4'] = data['actionTime_max_009']-data['actionTime_max_004']

data['actionTime_max_diff_9_m'] = data['actionTime_max_009']-data['userid_actionTime_max']
data['actionTime_max_diff_8_m'] = data['actionTime_max_008']-data['userid_actionTime_max']
data['actionTime_max_diff_7_m'] = data['actionTime_max_007']-data['userid_actionTime_max']
data['actionTime_max_diff_6_m'] = data['actionTime_max_006']-data['userid_actionTime_max']
data['actionTime_max_diff_5_m'] = data['actionTime_max_005']-data['userid_actionTime_max']
data['actionTime_max_diff_4_m'] = data['actionTime_max_004']-data['userid_actionTime_max']
data['actionTime_max_diff_3_m'] = data['actionTime_max_003']-data['userid_actionTime_max']
data['actionTime_max_diff_2_m'] = data['actionTime_max_002']-data['userid_actionTime_max']
data['actionTime_max_diff_1_m'] = data['actionTime_max_001']-data['userid_actionTime_max']



#actionType min time
feat = pd.DataFrame(pd.pivot_table(action,index=["userid"],columns='actionType',values=["actionTime"],aggfunc=min)).reset_index() 
feat.columns=["userid"]+["actionTime_min_00{0}".format(i) for i in range(1,10)]
data = data.merge(feat, on='userid', how='left')

data['actionTime_1_last'] = data['actionTime_max_001']-data['actionTime_min_001']
data['actionTime_2_last'] = data['actionTime_max_002']-data['actionTime_min_002']
data['actionTime_3_last'] = data['actionTime_max_003']-data['actionTime_min_003']
data['actionTime_4_last'] = data['actionTime_max_004']-data['actionTime_min_004']
data['actionTime_5_last'] = data['actionTime_max_005']-data['actionTime_min_005']
data['actionTime_6_last'] = data['actionTime_max_006']-data['actionTime_min_006']
data['actionTime_7_last'] = data['actionTime_max_007']-data['actionTime_min_007']
data['actionTime_8_last'] = data['actionTime_max_008']-data['actionTime_min_008']
data['actionTime_9_last'] = data['actionTime_max_009']-data['actionTime_min_009']

#actionType std time
feat = pd.DataFrame(pd.pivot_table(action,index=["userid"],columns='actionType',values=["actionTime"],aggfunc=np.std)).reset_index() 
feat.columns=["userid"]+["actionTime_std_00{0}".format(i) for i in range(1,10)]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(pd.pivot_table(action,index=["userid"],columns='actionType',values=["actionTime"],aggfunc=np.mean)).reset_index() 
feat.columns=["userid"]+["actionTime_mean_00{0}".format(i) for i in range(1,10)]
data = data.merge(feat, on='userid', how='left')



#time_rank actionTime
feat=pd.DataFrame(pd.pivot_table(action[['userid','time_rank','actionTime']], index='userid', columns='time_rank')).reset_index()
feat = feat.iloc[:,:26]
feat.columns=["userid"]+["actionTime_rank_{0}".format(i) for i in range(1,26)]

name = ["actionTime_rank_{0}".format(i) for i in range(1,26)]
feat['actionTime_rank_sum']  = feat[name].apply(lambda x: x.sum(),axis=1)
feat['actionTime_rank_std']  = feat[name].apply(lambda x: x.std(),axis=1)
feat['actionTime_rank_mean'] = feat[name].apply(lambda x: x.mean(),axis=1)
data = data.merge(feat, on='userid', how='left')


feat = pd.DataFrame(data['userid'])
feat['actionTime_rank_diff_1'] = data['actionTime_rank_1']-data['actionTime_rank_2']
feat['actionTime_rank_diff_2'] = data['actionTime_rank_2']-data['actionTime_rank_3']
feat['actionTime_rank_diff_3'] = data['actionTime_rank_3']-data['actionTime_rank_4']
feat['actionTime_rank_diff_4'] = data['actionTime_rank_4']-data['actionTime_rank_5']
feat['actionTime_rank_diff_5'] = data['actionTime_rank_5']-data['actionTime_rank_6']
feat['actionTime_rank_diff_6'] = data['actionTime_rank_6']-data['actionTime_rank_7']
feat['actionTime_rank_diff_7'] = data['actionTime_rank_7']-data['actionTime_rank_8']
feat['actionTime_rank_diff_8'] = data['actionTime_rank_8']-data['actionTime_rank_9']
feat['actionTime_rank_diff_9'] = data['actionTime_rank_9']-data['actionTime_rank_10']
feat['actionTime_rank_diff_10'] = data['actionTime_rank_10']-data['actionTime_rank_11']

name = ["actionTime_rank_diff_{0}".format(i) for i in range(1,10)]
feat['actionTime_rank_diff_sum']  = feat[name].apply(lambda x: x.sum(),axis=1)
feat['actionTime_rank_diff_std']  = feat[name].apply(lambda x: x.std(),axis=1)
feat['actionTime_rank_diff_mean'] = feat[name].apply(lambda x: x.mean(),axis=1)
data = data.merge(feat, on='userid', how='left')


In [16]:
#######################3、orderHistory
orderHistory['city']      = class_le.fit_transform(orderHistory['city'].values)
orderHistory['country']   = class_le.fit_transform(orderHistory['country'].values)
orderHistory['continent'] = class_le.fit_transform(orderHistory['continent'].values)

feat = pd.DataFrame(orderHistory.groupby(['userid'])[['orderType']].mean()).reset_index()
feat.columns=["userid"]+["orderType_mean"]
data = data.merge(feat, on='userid', how='left')


feat = pd.DataFrame(orderHistory.groupby(['userid'])[['orderType']].max()).reset_index()
feat.columns=["userid"]+["orderType_max"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(orderHistory.groupby(['userid'])[['orderType']].sum()).reset_index()
feat.columns=["userid"]+["orderType_sum"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(orderHistory.groupby(['userid'])[['orderType']].std()).reset_index()
feat.columns=["userid"]+["orderType_std"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(orderHistory.groupby(['userid'])[['orderTime']].count()).reset_index()
feat.columns=["userid"]+["orderHistory_cnt_all"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(orderHistory.groupby(['userid'])[['orderTime']].sum()).reset_index()
feat.columns=["userid"]+["orderTime_sum"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(orderHistory.groupby(['userid'])[['orderTime']].max()).reset_index()
feat.columns=["userid"]+["orderTime_max"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(orderHistory.groupby(['userid'])[['orderTime']].min()).reset_index()
feat.columns=["userid"]+["orderTime_min"]
data = data.merge(feat, on='userid', how='left')


feat = pd.DataFrame(orderHistory.groupby(['userid'])[['city']].mean()).reset_index()
feat.columns=["userid"]+["city_mean"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(orderHistory.groupby(['userid'])[['city']].std()).reset_index()
feat.columns=["userid"]+["city_std"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(orderHistory.groupby(['userid'])[['country']].mean()).reset_index()
feat.columns=["userid"]+["country_mean"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(orderHistory.groupby(['userid'])[['country']].std()).reset_index()
feat.columns=["userid"]+["country_std"]
data = data.merge(feat, on='userid', how='left')


feat = pd.DataFrame(orderHistory.groupby(['userid'])[['continent']].mean()).reset_index()
feat.columns=["userid"]+["ocontinent_mean"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(orderHistory.groupby(['userid'])[['continent']].std()).reset_index()
feat.columns=["userid"]+["ocontinent_std"]
data = data.merge(feat, on='userid', how='left')


data['orderTime_last'] = data['orderTime_max']-data['orderTime_min']

data['orderTime_max_year']  = data['orderTime_max'].fillna(0).apply(lambda x: time.localtime(int(x)).tm_year)
data['orderTime_max_month'] = data['orderTime_max'].fillna(0).apply(lambda x: time.localtime(int(x)).tm_mon)
data['orderTime_max_hour']  = data['orderTime_max'].fillna(0).apply(lambda x: time.localtime(int(x)).tm_hour)

feat = pd.DataFrame(pd.pivot_table(orderHistory,index=["userid"],columns='orderType',values=["orderTime"],aggfunc=len)).reset_index() 
feat.columns=["userid"]+["orderType_max_{0}".format(i) for i in range(0,2)]
data = data.merge(feat, on='userid', how='left')


In [17]:
################ 4、userComment
userComment['tags_long']  = userComment['tags'].fillna('a').apply(lambda x: len(x))
userComment['wrds_long']  = userComment['commentsKeyWords'].fillna('a').apply(lambda x: len(x))

feat = pd.DataFrame(userComment.groupby(['userid'])[['rating']].mean()).reset_index()
feat.columns=["userid"]+["rating_mean"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(userComment.groupby(['userid'])[['rating']].count()).reset_index()
feat.columns=["userid"]+["userComment_cnt_all"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(userComment.groupby(['userid'])[['tags_long']].mean()).reset_index()
feat.columns=["userid"]+["tags_long_mean"]
data = data.merge(feat, on='userid', how='left')

feat = pd.DataFrame(userComment.groupby(['userid'])[['wrds_long']].mean()).reset_index()
feat.columns=["userid"]+["wrds_long_mean"]
data = data.merge(feat, on='userid', how='left')

data['long_mean_all'] = data['tags_long_mean']+data['wrds_long_mean']


In [18]:
################ 组合特征
data['orderHistory_ctr'] = data['orderHistory_cnt_all']/data['action_cnt_all']
data['actiontime_orderTime_diff'] = data['userid_actionTime_max']-data['orderTime_max']
data['userComment_rate'] = data['userComment_cnt_all']/data['orderHistory_cnt_all']


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50383 entries, 0 to 50382
Columns: 282 entries, userid to userComment_rate
dtypes: float64(234), int64(39), uint8(9)
memory usage: 105.8 MB


In [20]:
data.head()

,userid,label,gender,province,age,gender__0,gender__1,gender__2,age__0,age__1,...,orderType_max_0,orderType_max_1,rating_mean,userComment_cnt_all,tags_long_mean,wrds_long_mean,long_mean_all,orderHistory_ctr,actiontime_orderTime_diff,userComment_rate
0,100000000013,0,2,0,1,0,0,1,0,1,...,2.0,NaN,4.0,1.0,1.0,19.0,20.0,0.013986,1278697.0,0.5
1,100000000111,0,0,1,5,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100000000127,0,0,1,5,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100000000231,0,2,4,2,0,0,1,0,0,...,NaN,NaN,5.0,1.0,9.0,10.0,19.0,NaN,NaN,NaN
4,100000000379,0,2,4,5,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
data.tail()

,userid,label,gender,province,age,gender__0,gender__1,gender__2,age__0,age__1,...,orderType_max_0,orderType_max_1,rating_mean,userComment_cnt_all,tags_long_mean,wrds_long_mean,long_mean_all,orderHistory_ctr,actiontime_orderTime_diff,userComment_rate
50378,114869469143,-1,1,28,5,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50379,114869567540,-1,2,10,3,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50380,114869867343,-1,0,12,2,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50381,114869967040,-1,2,19,5,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50382,114869967545,-1,0,0,5,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
data['label'].value_counts()

 0    33682
-1    10076
 1     6625
Name: label, dtype: int64

In [31]:
data.fillna(-1,inplace=True)

In [32]:
df_train_12 = data[data['label']> -1]
df_test_12 = data[data['label']== -1]

In [33]:
df_train_12.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40307 entries, 0 to 40306
Columns: 282 entries, userid to userComment_rate
dtypes: float64(234), int64(39), uint8(9)
memory usage: 84.6 MB


In [34]:
df_train_12.drop('label',axis=1,inplace=True)
df_test_12.drop('label',axis=1,inplace=True)


/home/xiaoran/Tools/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/xiaoran/Tools/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [35]:
df_train_12.to_csv('../data/dataSet/df_train_12.csv',index=False,index_label=False)
df_test_12.to_csv('../data/dataSet/df_test_12.csv',index=False,index_label=False)
